In [131]:
import pickle

with open("./dec_sample.pkl", "rb") as f:
    decile_sample = pickle.load(f)

In [132]:
import pprint

pp = pprint.PrettyPrinter()
pp.pprint(decile_sample)

{0: {'Independent': 6.645139583565834,
     'MAE': 6.602579969147039,
     'OSError': 7.295727149706984,
     'Sal': 7.072583598392774,
     'basin': 6.22528573800557,
     'methods': 3.586228408390312,
     'processed': 5.621750716135312,
     'resulting': 4.424047524822972,
     'traverse': 7.008045077255203,
     'модель': 5.888813501384357},
 1: {'Suspects': 7.988874330266929,
     'contemporary': 7.701192257815148,
     'discount_factor': 7.988874330266929,
     'firstnum': 7.478048706500939,
     'nonzeroy': 7.478048706500939,
     'scale_color_brewer': 7.701192257815148,
     'train_lgb': 7.8347236504396705,
     'unlearned': 7.8347236504396705,
     'zero_with_resv': 7.8347236504396705,
     'Пока': 7.583409222158765},
 2: {'Gott': 8.171195887060883,
     'Remarque': 8.171195887060883,
     'delta_price1': 8.394339438375093,
     'dirpath': 8.171195887060883,
     'min_list_1': 8.394339438375093,
     'numRows此圖合併了幾行': 8.171195887060883,
     'occupants': 8.394339438375093,
   

In [133]:
def report(t):   
    pos = !grep -il "\b{t}\b" extracts/*
    pos = set([basename(f) for f in pos])
    res = set([basename(r['key']) for r in b.search(t, size=10000) if basename(r['key']).startswith("nb_")])

    tps = res.intersection(pos)
    fps = res.difference(pos)
    fns = pos.difference(res)

    return {
        'ground_truth': pos,
        'search_results': res,
        'true_positives': tps,
        'false_positives': fps,
        'false_negatives': fns,
    }

In [122]:
from os.path import basename

import t4

b = t4.Bucket("s3://t4-staging")

stats = {}
for d in range(0,10):
    print(f"= Decile {d} =")
    stats[d] = {}
    for t in decile_sample[d].keys():
        rep = report(t)
        
        tps = len(rep['true_positives'])
        fps = len(rep['false_positives'])
        fns = len(rep['false_negatives'])
        
        pdenom = tps + fps
        rdenom = tps + fns
        
        stats[d][t] = {
            'precision': tps / pdenom if pdenom else None,
            'recall': tps / rdenom if rdenom else None,
            'tps': tps,
            'fps': fps,
            'fns': fns

        }

= Decile 0 =
= Decile 1 =
= Decile 2 =
= Decile 3 =
= Decile 4 =
= Decile 5 =
= Decile 6 =
= Decile 7 =
= Decile 8 =
= Decile 9 =


In [154]:
pp.pprint(stats[5])

{'amp_t': {'fns': 0, 'fps': 0, 'precision': 1.0, 'recall': 1.0, 'tps': 2},
 'sel_train': {'fns': 6, 'fps': 0, 'precision': None, 'recall': 0.0, 'tps': 0},
 'selection_numbers': {'fns': 0,
                       'fps': 0,
                       'precision': 1.0,
                       'recall': 1.0,
                       'tps': 1},
 'set_y': {'fns': 1, 'fps': 0, 'precision': None, 'recall': 0.0, 'tps': 0},
 '每个子分类器都会对一个样本给出一个预测结果': {'fns': 0,
                           'fps': 145,
                           'precision': 0.00684931506849315,
                           'recall': 1.0,
                           'tps': 1},
 '熔点': {'fns': 0,
        'fps': 51,
        'precision': 0.019230769230769232,
        'recall': 1.0,
        'tps': 1},
 '默认值为0': {'fns': 0,
           'fps': 5126,
           'precision': 0.0001950458357714063,
           'recall': 1.0,
           'tps': 1},
 '들어갈수': {'fns': 0, 'fps': 0, 'precision': 1.0, 'recall': 1.0, 'tps': 1},
 '맥락이': {'fns': 0, 'fps': 0, 'precisi

In [160]:
pr = {}
for d in range(0,10):
    s = stats[d]
    csize = sum([s[t]['tps'] for t in s])
    wprecision = [s[t]['tps']*s[t]['precision'] for t in s if s[t]['precision']]
    wrecall = [s[t]['tps']*s[t]['recall'] for t in s if s[t]['recall']]

    pr[d] = {
        'precision': sum(wprecision)/csize,
        'recall': sum(wrecall)/csize,
        'docs': csize
    }

In [161]:
pr

{0: {'precision': 0.9991499696417728,
  'recall': 0.9983338742832413,
  'docs': 1170},
 1: {'precision': 1.0, 'recall': 1.0, 'docs': 84},
 2: {'precision': 0.9159180740208409, 'recall': 1.0, 'docs': 46},
 3: {'precision': 1.0, 'recall': 1.0, 'docs': 23},
 4: {'precision': 0.8909276248725789, 'recall': 1.0, 'docs': 18},
 5: {'precision': 0.6695861255705593, 'recall': 1.0, 'docs': 9},
 6: {'precision': 0.9007999999999999, 'recall': 1.0, 'docs': 10},
 7: {'precision': 1.0, 'recall': 1.0, 'docs': 32},
 8: {'precision': 1.0, 'recall': 1.0, 'docs': 82},
 9: {'precision': 1.0, 'recall': 1.0, 'docs': 18}}

In [166]:
table = [
    (pr[d]['precision'], pr[d]['recall'], pr[d]['docs']) for d in pr
]

import pandas as pd

df = pd.DataFrame(table, columns=['precision', 'recall', 'docs'])
df

,precision,recall,docs
0,0.999150,0.998334,1170
1,1.000000,1.000000,84
2,0.915918,1.000000,46
3,1.000000,1.000000,23
4,0.890928,1.000000,18
5,0.669586,1.000000,9
6,0.900800,1.000000,10
7,1.000000,1.000000,32
8,1.000000,1.000000,82
9,1.000000,1.000000,18


In [173]:
precision = (df['precision']*df['docs']).sum() / df['docs'].sum()

In [172]:
recall = (df['recall']*df['docs']).sum() / df['docs'].sum()

In [176]:
f1 = 2*(precision * recall)/(precision + recall)
f1

0.9957215110540265

In [153]:
pr

{0: {'precision': 0.9991499696417728,
  'recall': 0.9983338742832413,
  'docs': 1170},
 1: {'precision': 1.0, 'recall': 1.0, 'docs': 84},
 2: {'precision': 0.9159180740208409, 'recall': 1.0, 'docs': 46},
 3: {'precision': 1.0, 'recall': 1.0, 'docs': 23},
 4: {'precision': 0.8909276248725789, 'recall': 1.0, 'docs': 18},
 5: {'precision': 0.6695861255705593, 'recall': 1.0, 'docs': 9},
 6: {'precision': 0.9007999999999999, 'recall': 1.0, 'docs': 10},
 7: {'precision': 1.0, 'recall': 1.0, 'docs': 32},
 8: {'precision': 1.0, 'recall': 1.0, 'docs': 82},
 9: {'precision': 1.0, 'recall': 1.0, 'docs': 18}}

In [ ]:
for d in range(0,10):
    s = pr[d]
    wpr = [x['precision'] for x in pr]